# Gabor's Neighborhoods in Toronto project  


## Part 1: Wikipedia data pull and manipulation

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Import done.')

Import done.


In [2]:
# read in the html from website
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
d = table[0]

In [3]:
# create a dataframe
data = d[['Postal code','Borough','Neighborhood']]
data.head(3)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods


In [4]:
# Let's see how many lines are there in total
data.shape

(180, 3)

In [5]:
# Let's see how many "Not assigned" has to be dropped
data['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [6]:
# Filter out the 77 lines with "Not assigned"
condition = data['Borough'] != 'Not assigned'
data2 = data[condition]
data2.head(3)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


**COMMENT:**  
These below actions I could not perform as M5A was not listed twice and after the Not assigned Borough removal there were no blank Neighborhoods at all. 

1. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
# data2['Neighborhood'].value_counts()                 un-hashtag to see there is no Not assigned neighborhood

In [8]:
# Double-check to see if 180-77= 103 lines remained
data2.shape

(103, 3)

## Part 2: Adding geo data

In [47]:
# read in the geo data from website
geos = pd.read_csv('http://cocl.us/Geospatial_data')
geos.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [48]:
# Changing the capital C to lower case c in the column name
geos['Postal code']=geos['Postal Code']
geos.drop("Postal Code", axis = 1, inplace=True)
geos.head()

,Latitude,Longitude,Postal code
0,43.806686,-79.194353,M1B
1,43.784535,-79.160497,M1C
2,43.763573,-79.188711,M1E
3,43.770992,-79.216917,M1G
4,43.773136,-79.239476,M1H


In [49]:
# merging the tables based on the postal code
merged = pd.merge(data2,geos, on='Postal code')
merged.head(3)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636


*Explore and cluster the neighborhoods in Toronto. **You can decide to work with only boroughs that contain the word Toronto** and then replicate the same analysis we did to the New York City data.* 

In [50]:
# Filter out the Boroughs with "Toronto"
condition = merged['Borough'].str.contains('Toronto')    
merged2 = merged[condition]
merged2.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [51]:
# See the size of the df
merged2.shape

(39, 5)

## Part 3: Clustering Toronto

In [14]:
# if needed, uncheck

!pip install geopy        
!pip install wget

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

In [53]:
# Run k-means to cluster the neighborhood, 
# set number of clusters
kclusters = 5

merged2_clustering = merged2[['Latitude','Longitude']]
                                           
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged2_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 4, 0, 0, 3, 0, 1], dtype=int32)

In [61]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [62]:
# Visualize the resulting clusters, create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged2['Latitude'], merged2['Longitude'], merged2['Neighborhood'], merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters